# Inference

In [1]:
# Ensure local package is importable in this notebook (editable install) and add project root to sys.path.
from pathlib import Path
import sys

project_root = str(Path("..").resolve())
if project_root not in sys.path:
	sys.path.insert(0, project_root)

from src.models.ebm_predictor import EBMPredictor

In [2]:
predictor = EBMPredictor(
    model_path="/home/chlab/flync/final_train_artifacts/FINAL_flync_ebm_model.pkl",
    schema_path="/home/chlab/flync/final_train_artifacts/flync_model_schema.json",
    check_value_ranges=True
)

In [3]:
import pandas as pd

inference = pd.read_parquet("cleaned_dataset_redux.parquet")
res = pd.read_parquet('features.parquet').set_index('index')

inference.shape, res.shape

((18962, 731), (20759, 43))

In [4]:
res_cols = ['transcript_id', 'chromosome', 'start', 'end', 'ss_sequence', 'ss_structure']

res = res[[c for c in res.columns if c in res_cols]]

In [5]:
res

,chromosome,start,end,ss_sequence,ss_structure
index,,,,,
MSTRG.2.1,chr2L,9837,18570,CACACUAGACCCACCGAUCGCGACUUUGUGUGUGUUAAAAUGACGC...,....((((......((((((((((..(((((((((((...))))))...
MSTRG.2.13,chr2L,9842,15711,ACGGCACAGCAUGGCUUUCCUCAUAAGCCUUCGGCUCUUGCGUAUG...,((((((((((..(((((.......)))))...(((((((..(((((...
MSTRG.2.15,chr2L,10953,15711,ACGGCACAGCAUGGCUUUCCUCAUAAGCCUUCGGCUCUUGCGUAUG...,((((((((((..(((((.......)))))...(((((((..(((((...
MSTRG.2.16,chr2L,15987,19879,GUAAAACAAACAAUUUUCUCUUUCCGUUCUAUUUUUUUUUUUUUAA...,.........................((((......(((((((((((...
MSTRG.2.17,chr2L,16199,21424,AAAUAUACCUUCUAAAAAAUAUAUGAUUUUUUCGAUAGAGAUACAC...,.........((((...........((((((.(((((((((((((.(...
...,...,...,...,...,...
MSTRG.25380.1,chrY,3637823,3640009,UUGUGUCCUUCUUUUCUAUUAAUUCUGCCCAUUUGGGAGCGCCGGC...,(((((.(((((((((......(((((.(((...(((((((...(((...
MSTRG.25381.1,chrY,3648692,3649112,CAGUAUAUGUGUACAAAAAGAACGAAUAAAGAACGGUAUGGAUGCA...,(((.((((.(((.((((...............(((((((.(.((((...
MSTRG.25382.2,chrY,3665131,3666483,GAAGUGGAGCAGGUGUAGAAAUAGGAAGAGAACUAGGGGUAAAUAG...,...(((((((.((((((((..(((((((.(((...(((((...(((...


In [6]:
inference["proba"] = pd.DataFrame(predictor.predict_proba(inference, validate=False)[:, 1], index=inference.index)

In [7]:
inference["pred"] = predictor.predict(inference.drop(columns=["proba"]), validate=False)

In [8]:
res = res.merge(inference[['proba', 'pred']], how='right', left_index=True, right_index=True)

In [9]:
import pandas as pd
from pathlib import Path

base_path = '/mnt/d/flync_final_runs/Vu_GSE199164/run_2_used_for_qPCR/cov'

sources = Path(base_path).glob('*/t_data.ctab')

paths = {p.parent.name: p for p in sources}

def parse_ctabs(paths: dict[str, Path]) -> pd.DataFrame:
    dfs = []
    for name, path in paths.items():
        df = pd.read_csv(path, sep='\t', usecols=['t_name', 'FPKM'])
        df = df.rename(columns={'FPKM': f'fpkm_{name.lower()}'})
        df.drop_duplicates(subset=['t_name'], inplace=True)
        dfs.append(df)
    merged_df = dfs[0]
    for df in dfs[1:]:
        merged_df = merged_df.merge(df, on='t_name', how='outer')
    return merged_df

def fpkm_stats(fpkm_df: pd.DataFrame) -> pd.DataFrame:
    fpkm_values = fpkm_df.drop(columns=['t_name'])
    fpkm_df['fpkm_mean'] = fpkm_values.mean(axis=1)
    fpkm_df['fpkm_median'] = fpkm_values.median(axis=1)
    fpkm_df['fpkm_std'] = fpkm_values.std(axis=1)
    return fpkm_df

fpkm = fpkm_stats(parse_ctabs(paths))

In [10]:
fpkm

,t_name,fpkm_srr18432803,fpkm_srr18432804,fpkm_srr18432805,fpkm_srr18432812,fpkm_srr18432813,fpkm_srr18432814,fpkm_mean,fpkm_median,fpkm_std
0,FBgn0013687_df_pt,0.024063,0.039446,0.087131,0.056102,0.000000,0.026194,0.038823,0.032820,0.030057
1,FBti0015567-RA,1.750656,1.648326,2.431020,1.294828,0.609853,0.822420,1.426184,1.471577,0.665208
2,FBti0018861-RA,3.286168,2.879346,4.526400,2.045450,1.571053,1.907091,2.702585,2.462398,1.099374
3,FBti0018862-RA,3.867935,3.465611,5.339834,2.466703,1.902489,2.441864,3.247406,2.966157,1.255239
4,FBti0018865-RA,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...
60044,RR51000_transposable_element-RA,4.589861,4.249411,3.535903,3.249276,2.537714,3.931326,3.682249,3.733614,0.738419
60045,RR51001_transposable_element-RA,1.441380,0.576218,0.929710,0.356585,0.316498,0.213489,0.638980,0.466402,0.468421
60046,RR51007_transposable_element-RA,7.091676,5.514330,7.038363,7.390827,8.942161,9.874952,7.642051,7.241251,1.544914
60047,RR51048_transposable_element-RA,6.321496,6.252961,11.994285,4.202961,1.392507,2.329779,5.415665,5.227961,3.794179


In [11]:
res = res.merge(fpkm, how='left', left_index=True, right_on='t_name').drop(columns=['t_name'])

In [12]:
res

,chromosome,start,end,ss_sequence,ss_structure,proba,pred,fpkm_srr18432803,fpkm_srr18432804,fpkm_srr18432805,fpkm_srr18432812,fpkm_srr18432813,fpkm_srr18432814,fpkm_mean,fpkm_median,fpkm_std
39137,chr2L,5494872,5497738,UCCAUAGACAGUCGGCCCACGCCUUUGGUAGUCAAAUGAAAUCAGC...,.........((.((((.(((((((.(((....((((((..((((((...,6.319258e-06,False,1.989737,1.866601,3.553927,0.759103,0.651369,0.339000,1.526623,1.312852,1.200469
39138,chr3L,1167797,1169634,UUGCUGGCUAAUUUCACGCCGAAUCAUCCGCCUGCCAAAGGAUGCA...,..(((.((((((((...(((((((((((.((((((((..((((.(....,9.503701e-06,False,7.984267,6.906512,12.938016,5.427372,2.001333,4.927536,6.697506,6.166942,3.672696
39139,chr3L,1182069,1182834,GGAAACACUCGCUCCUUGGCCAGGACUUUCAGCCAGCUUUUUCGCA...,.......((.(((....))).))....((((((((((............,6.965118e-08,False,1.925921,2.370615,4.989229,0.572829,0.339874,1.101849,1.883386,1.513885,1.708581
39140,chr3L,1186455,1186968,UUUUUUUUUUUUUACCCUCCACUUGGAAAAUAAAAUGAUGGGAAUG...,.................(((....))).......(((((((..(((...,2.099218e-06,False,1.519468,2.807598,3.983369,0.151821,0.000000,0.000000,1.410376,0.835645,1.682065
39141,chr3L,1196207,1199339,AGUUUGGCUUACCAGCGAGAGAGAGAGAGAGAAAUAAAGAGAGAGC...,.....(((((....((..(((((((.(((...((((......((((...,1.201409e-07,False,11.908412,20.214312,1.093409,11.357080,10.823297,6.208575,10.267514,11.090189,6.383832
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59891,chr3L,1073769,1076351,CAAUAUGUGUAUUUAAAUGCAAACAUAAAAUUUAUGUCAAAUGUUU...,...(((((((........((((((((..............))))))...,2.075910e-05,False,4.577745,3.848118,9.345529,4.564241,0.649505,2.065856,4.175166,4.206180,2.967576
59892,chr3L,1118913,1119342,GCACUUGCCCAAUUGGGCAGCAGUGAGGUCAUUAUUGCAACUAUUG...,.(((((((((....)))))..)))).((.....((((((((...((...,1.365000e-07,False,0.000000,0.860304,6.268565,0.000000,0.202025,0.000000,1.221816,0.101013,2.494748
59893,chr3L,1130291,1134454,AGCUGGGGGAGCAGCCACAGGAGCAGCAAUCAUACGAUAUUUUCAG...,.((((((((((.(((((.((((((...........((((.((((.....,5.795971e-04,False,2.097524,1.944203,4.318320,0.925513,0.506725,1.206742,1.833171,1.575473,1.358962
59894,chr3L,1136617,1139194,UUUUUUUUUUGUCAUUCUGCAUGUUGUAUAUACAACAGACUUGGUU...,......(((((((.....(..((((((....))))))..)((((((...,5.976679e-07,False,2.471525,0.480314,2.983271,0.124168,0.299991,0.079008,1.073046,0.390153,1.299394


In [13]:
res.to_excel("flync_vu_gse199164_inference_results.xlsx", index=True)